In [2]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

C:\Users\Yugant Manekar\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class PlannerState(TypedDict):
    messages : Annotated[List[HumanMessage | AIMessage],"the message in the conversation"]
    city:str
    interests:List[str]
    itinerary:str

In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    temperature = 0,
    groq_api_key = "Your API Key",
    model_name = "Any Model Name"
)

In [7]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's intersests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])

In [17]:
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip: ")
    user_message = input("Your Input: ")
    return {
        **state,
        "city":user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter your interest for the trip to: {state['city']} (comma-seperated): " )
    user_message = input("Your Input: ")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"Creating an itinerary for {state['city']} based on interests : {', '.join(state['interests'])}")
    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], interests=','.join(state['interests'])))
    print("\n Final Itinerary: ")
    print(response.content)
    return {
        **state,
        "messages" : state["messages"] + [AIMessage(content=response.content)],
        "itinerary" : response.content,
    }

In [18]:
workflow = StateGraph(PlannerState)

workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_city")

workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "create_itinerary")
workflow.add_edge("create_itinerary", END)

app = workflow.compile()

In [19]:
def travel_planner(user_request: str):
    print(f"Initial Request: {user_request}\n")
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",
        "interests": [],
        "itinerary": "",
    }
    for output in app.stream(state):
        pass

In [20]:
user_request = "I want to plan a day trip"
travel_planner(user_request)

Initial Request: I want to plan a day trip

Please enter the city you want to visit for your day trip: 


Your Input:  Uttarakhand


Please enter your interest for the trip to: Uttarakhand (comma-seperated): 


Your Input:  kedarnath,badrinath,tungnath


Creating an itinerary for Uttarakhand based on interests : kedarnath, badrinath, tungnath

 Final Itinerary: 
For a day trip to Uttarakhand focusing on Kedarnath, Badrinath, and Tungnath, here's a brief itinerary:

* 4:00 AM: Start the day with an early morning drive to Guptkashi (approx. 1 hour from Sonprayag) for a quick breakfast.
* 6:00 AM: Proceed to Gaurikund (approx. 1.5 hours from Guptkashi) and begin the trek to Kedarnath (14 km, approx. 4-5 hours).
* 11:00 AM: Visit the Kedarnath Temple and explore the surrounding area.
* 1:00 PM: Have lunch at Kedarnath and begin the descent to Gaurikund.
* 4:00 PM: Drive to Tungnath (approx. 2 hours from Gaurikund) and visit the Tungnath Temple, the highest Shiva temple in the world.
* 6:30 PM: Drive to Badrinath (approx. 3 hours from Tungnath) and visit the Badrinath Temple.
* 8:30 PM: Return to your starting point, concluding the day trip.

Note: This itinerary is quite ambitious and may not be feasible in a single day, considering the lo

In [23]:
import gradio as gr
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

class PlannerState(TypedDict):
    messages : Annotated[List[HumanMessage | AIMessage], "the message in the conversation"]
    city: str
    interests: List[str]
    itinerary: str

llm = ChatGroq(
    temperature = 0,
    groq_api_key = "Your API Key",
    model_name = "Any Model name"
)

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])

def input_city(city: str, state: PlannerState) -> PlannerState:
    return {
        **state,
        "city": city,
        "messages": state['messages'] + [HumanMessage(content=city)]
    }

def input_interest(interests: str, state: PlannerState) -> PlannerState:
    return {
        **state,
        "interests": [interest.strip() for interest in interests.split(",")],
        "messages": state['messages'] + [HumanMessage(content=interests)]
    }

def create_itinerary(state: PlannerState) -> str:
    response = llm.invoke(
        itinerary_prompt.format_messages(
            city = state['city'],
            interests = ','.join(state['interests'])
        )
    )
    state["itinerary"] = response.content
    state["messages"] += [AIMessage(content=response.content)]
    return response.content

def travel_planner(city: str, interests: str):
    state = {
        "messages": [],
        "city": "",
        "interests": [],
        "itinerary": "",
    }

    state = input_city(city, state)
    state = input_interest(interests, state)

    itinerary = create_itinerary(state)

    return itinerary

interface = gr.Interface(
    fn = travel_planner,
    inputs = [
        gr.Textbox(label="Enter the city for your day trip"),
        gr.Textbox(label="Enter your interests (comma-separated)"),
    ],
    outputs = gr.Textbox(label="Generated Itinerary"),
    title = "Travel Itinerary Planner",
    description = "Enter a city and your interests to generate a personalized day trip itinerary."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
